In [1]:
from txtai.embeddings import Embeddings
from txtai.app import Application

In [6]:
config = Application.read("./app.yml")
embeddings = Embeddings(config["embeddings"], content=True)

# Can also use the config programmatically
#embeddings = Embeddings({
#    "path": "sentence-transformers/all-MiniLM-L6-v2",
#    "backend": "qdrant.Qdrant",
#})


In [ ]:
from txtai.pipeline import LLM

# Create LLM
llm = LLM("TheBloke/Mistral-7B-OpenOrca-AWQ")

In [ ]:
from txtai.pipeline import Textractor

# Document text extraction, split into paragraphs
textractor = Textractor(paragraphs=True)

def stream(path):
  for f in sorted(os.listdir(path)):
    fpath = os.path.join(path, f)

    # Only accept documents
    if f.endswith(("docx", "xlsx", "pdf")):
      print(f"Indexing {fpath}")
      for paragraph in textractor(fpath):
        yield paragraph


In [ ]:
embeddings.index(stream("docs"))

In [ ]:
def prompt_llm(question, text):
  prompt = f"""<|im_start|>system
  You are a friendly assistant. You answer questions from users.<|im_end|>
  <|im_start|>user
  Answer the following question using only the context below. Only include information specifically discussed.

  question: {question}
  context: {text} <|im_end|>
  <|im_start|>assistant
  """

  return llm(prompt, maxlength=4096, pad_token_id=32000)

def context(prompt, ann_embeddings):
  context =  "\n".join(x["text"] for x in )
  return context

def rag(prompt):
  ann_embeddings = embeddings.search(prompt)
  return prompt_llm(prompt, context(prompt, ann_embeddings))

In [ ]:
result = rag("When was the BLIP model added for image captioning?")
print(result)

# Citations in RAG

We can also build pipelines that provide citations for the results

In [ ]:
from txtai.pipeline import Extractor

# Extractor prompt
def prompt(question):
  return [{
    "query": question,
    "question": f"""
Answer the following question using only the context below. Only include information specifically discussed.

question: {question}
context:
"""
}]

# Create LLM with system prompt template
llm = LLM("TheBloke/Mistral-7B-OpenOrca-AWQ", template="""<|im_start|>system
You are a friendly assistant. You answer questions from users.<|im_end|>
<|im_start|>user
{text} <|im_end|>
<|im_start|>assistant
""")

# Create extractor instance
extractor = Extractor(embeddings, llm, output="reference")

In [ ]:
result = extractor(prompt("What version of Python is supported?"), maxlength=4096, pad_token_id=32000)[0]
print("ANSWER:", result["answer"])
print("CITATION:", embeddings.search("select id, text from txtai where id = :id", limit=1, parameters={"id": result["reference"]}))